In [1]:
with open('NER_v1.0.json') as file:
    content = file.readlines()

In [2]:
content[0]

'{"text": "Харин \\"Тавантолгой\\" ХК-ийн уурхайчид 2017 онд 141 тэрбум төгрөгийн ашигтай ажиллаад удахгүй хувьцаа эзэмшигчдийнхээ хурлыг хийж ногдол ашгаа хуваарилах юм байна.", "labels": [[7, 18, "ORG"]]}\n'

In [3]:
from ast import literal_eval

In [4]:
content = [literal_eval(i) for i in content ]

In [5]:
content[-1]

{'text': 'Яагаад ийм яриа гарах болсон нөхцөл нь Улсын бүртгэлийн багц хууль батлагдсантай холбоотой гэж бодож байна.',
 'labels': [[39, 66, 'MISC']]}

In [6]:
content[-1]['labels'][0][:-1]

[39, 66]

In [7]:
len(content)

10162

In [8]:
content[-1]['text'][39:66].split()

['Улсын', 'бүртгэлийн', 'багц', 'хууль']

In [9]:
content[1000]['text'].split()

['Тэгвэл',
 'Лонзо',
 'хэдхэн',
 'хоногийн',
 'өмнө',
 '"Кайли',
 'Кузма"',
 'нэртэй',
 'шинэ',
 'дууныхаа',
 'клипийг',
 'цацсан',
 'билээ.']

In [14]:
converted_sents = []
for con in content:
    if len(con['labels']) > 0:
        new_content = {}
        for label in con['labels']:
            start = label[:-1][0]
            end = label[:-1][1]
            if len(con['text'][start:end].split()) > 1: 
                replacer = []
                for idx, sub in enumerate(con['text'][start:end].split()):
                    if idx == 0:
                        replacer.append(sub + '_B-' +label[2])
                    else:
                        replacer.append(sub + '_I-' +label[2])
                new_content['{}:{}'.format(start, end)] = ' '.join(replacer)
            else:
                new_content['{}:{}'.format(start, end)] = con['text'][start:end] + '_B-' +label[2]
        sents = []
        for k,v in new_content.items():
            start = int(k.split(':')[0])
            end = int(k.split(':')[1])
            sents.append(con['text'].replace(con['text'][start:end], v))
        converted_sents.append(sents)
    else:
        converted_sents.append([con['text']])

In [15]:
len(converted_sents)

10162

In [16]:
converted_sents[0]

['Харин "Тавантолгой_B-ORG" ХК-ийн уурхайчид 2017 онд 141 тэрбум төгрөгийн ашигтай ажиллаад удахгүй хувьцаа эзэмшигчдийнхээ хурлыг хийж ногдол ашгаа хуваарилах юм байна.']

In [3]:
import pandas as pd

In [18]:
left = pd.DataFrame(converted_sents).iloc[:,0].fillna(' ').apply(lambda x: x.split())

In [19]:
right = pd.DataFrame(converted_sents).iloc[:,1].fillna(' ').apply(lambda x: x.split())

In [20]:
def singlify(row):
    combined = []
#     print(row[0], row[1])
    if row[1] == []:
        return ' '.join(row[0])
    else:
        for i,j in zip(row[0], row[1]):
            if len(i) > len(j):
                combined.append(i)
            else:
                combined.append(j)
        return ' '.join(combined)

In [21]:
df = pd.DataFrame([left, right]).transpose().apply(lambda x: singlify(x), axis=1)

In [22]:
df.head()[4]

'Надад одоогоор УИХ_B-ORG-ын Тамгын_B-ORG газраас_I-ORG ээлжит бус чуулганыг хуралдуулахтай холбоотой мэдэгдэл ирээгүй байна.'

In [23]:
import re

In [24]:
pattern = '_B-ORG|_I-ORG|_B-LOC|_I-LOC|_B-MISC|_I-MISC|_B-PER|_I-PER'

In [31]:
with open('mn-ner-conll2003.txt', 'w') as file:
    for idx, sent in df.items():
        for word in sent.split():
            if any(x in word for x in ['LOC', 'MISC', 'ORG', 'PER']):
                if any(word.endswith(x) for x in ['LOC', 'MISC', 'ORG', 'PER']):
                    file.write(f"{word.split('_')[0]} O O {word.split('_')[1]}\n")
                else:  # wrong labeled tokens
                    tag = re.findall(pattern, word)[0][1:]
                    string = ''.join(re.split(pattern, word))
                    file.write(f"{string} O O {tag}\n")
            else:
                file.write(f"{word} O O O\n")
        file.write(f"\n")

In [27]:
# compatible with - https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/ - DOTD
Word = []
Tag = []
IDS = []
for idx, sent in df.items():
    for word in sent.split():
        if any(x in word for x in ['LOC', 'MISC', 'ORG', 'PER']):
            if any(word.endswith(x) for x in ['LOC', 'MISC', 'ORG', 'PER']):
                Word.append(word.split('_')[0])
                Tag.append(word.split('_')[1])
                IDS.append(idx)
            else:  # wrong labeled tokens
                tag = re.findall(pattern, word)[0][1:]
                word = ''.join(re.split(pattern, word))
                Word.append(word)
                Tag.append(tag)
                IDS.append(idx)
        else:
            Word.append(word)
            Tag.append('O')
            IDS.append(idx)

In [28]:
len(Word), len(Tag), len(IDS)

(158600, 158600, 158600)

In [29]:
DOTD_df = pd.DataFrame({'Word':Word, 'Tag':Tag, 'Sentence #':IDS})

In [30]:
DOTD_df.to_csv('ner_dataset.csv', index=False)

## Train Test Valid split

In [4]:
DOTD_df = pd.read_csv('ner_dataset.csv')

In [5]:
DOTD_df.head()

,Word,Tag,Sentence #
0,Харин,O,0
1,"""Тавантолгой""",B-ORG,0
2,ХК-ийн,O,0
3,уурхайчид,O,0
4,2017,O,0


In [6]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [7]:
getter = SentenceGetter(DOTD_df)

In [8]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Харин "Тавантолгой" ХК-ийн уурхайчид 2017 онд 141 тэрбум төгрөгийн ашигтай ажиллаад удахгүй хувьцаа эзэмшигчдийнхээ хурлыг хийж ногдол ашгаа хуваарилах юм байна.'

In [9]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
print(labels[0])

['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [12]:
len(X_train), len(X_val), len(X_test)

(8230, 915, 1017)

In [13]:
y_train[0]

['B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [21]:
X_train[0], X_val[0], X_test[0]

('Д.Гантулга гишүүн улайм цайм "Тэндээс би бандаашаа өмсөөд гарсан" гэх мэдээллийг олон нийтийн өмнө хийчихээд байхад шударга шүүх юу ч болоогүй мэт хаяхгүй байлгүй дээ.',
 'УАНС-аас олгох буцалтгүй тусламжийн хөрөнгийг уур амьсгалын өөрчлөлтийн нөлөөг бууруулах болон уур амьсгалын өөрчлөлтөд дасан зохицоход чиглэсэн оновчтой шийдлүүдийг нэвтрүүлсэн барилгын компаниудтай хийх үр дүнд суурилсан буцалтгүй тусламжид зарцуулна.',
 'Нэгдүгээрт 39 дүгээр сургуулиас "Хүчит шонхор" захын уулзвар хүртэлх 800 метр замын ажил.')

In [27]:
with open('train.txt', 'w') as train:
    for sentence, label in zip(X_train, y_train):
        for word, lab in zip(sentence.split(), label):
            train.write(f'{word} O O {lab}\n')
        train.write(f"\n")

In [28]:
with open('test.txt', 'w') as test:
    for sentence, label in zip(X_test, y_test):
        for word, lab in zip(sentence.split(), label):
            test.write(f'{word} O O {lab}\n')
        test.write(f"\n")

In [29]:
with open('valid.txt', 'w') as valid:
    for sentence, label in zip(X_val, y_val):
        for word, lab in zip(sentence.split(), label):
            valid.write(f'{word} O O {lab}\n')
        valid.write(f"\n")